In [1]:
from bs4 import BeautifulSoup
import requests
import time
import csv
import re
import pandas as pd
import requests
import numpy as np
from __future__ import unicode_literals 
from bosonnlp import BosonNLP
nlp = BosonNLP('Yy7Tpb83.27824.Zdrfgp6JtKTf')
import os
import random
import nltk
Porter = nltk.stem.PorterStemmer()
import pickle
from langconv import Converter

In [2]:
def readfile(filename,method='utf-8'):
    rawlist = []
    with open(filename,'r+', encoding=method) as csvfile:
        rows = csv.reader(csvfile)
        for row in rows:
            rawlist.append(row)
    return rawlist

def writefile(filename,inputlist,method='utf-8'):
    with open(filename,'a+', encoding=method) as csvfile:
        writer = csv.writer(csvfile, delimiter=',')
        for row in inputlist:
            writer.writerow(row)
def regex(a):
    reg = []
    a = re.sub('[,、：；！？，。－"\']','#',a)
    a = re.sub('[「」(){}:;!?]','',a)
    a = re.sub('\n','',a)
    #a = re.sub('[(),{}\u3000\*\|=-\[\]\n.]','',a)
    a = re.sub(u"\\（.*?\\）|\\『.*?』|\\「.*?」|\\〔.*?〕|\\[.*?]|\\〈.*?〉", '', a)
    reg = a.split('#')
    return reg

def no_English(a):
    a = re.sub('[a-zA-Z]','X',a)
    if 'X' in a:
        return False
    else:
        return True
def to_mandarin(inputstr=''):
    output = Converter('zh-hant').convert(inputstr)
    return(output)
def sentencelize(inputstr=''):
    output = regex(to_mandarin(inputstr))
    for i in range(0,len(output)-1):
        if len(output[i])<=3:
            output[i+1] = output[i] + output[i+1]
    return output
def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

In [84]:
with open('Paper_Dmpairs_ENG.pickle','rb') as f:
    AiritiSource = pickle.load(f)
AiritiRaw = []
for doc in AiritiSource:
    for pair in doc:
        AiritiRaw.append(pair)
len(AiritiRaw)

8704

In [85]:
cboson = []
tag_boson = []
tmp = ''
start = 0
for i in range(0,len(AiritiRaw)):
    try:
        result = nlp.tag(AiritiRaw[i][0])
        for word in result[0]['word']:
            tmp = tmp+word+'#'
        cboson.append(tmp)
        tmp = ''
        for tag in result[0]['tag']:
            tmp = tmp+tag+'#'
        tag_boson.append(tmp)
        tmp = ''
    except:
        print('summary',i,'does not work.')
    start = start+1
    if(i%100==0):
        print('finish',start,'sentences')
        #print(result)

finish 1 sentences
finish 101 sentences
finish 201 sentences
finish 301 sentences
finish 401 sentences
finish 501 sentences
finish 601 sentences
finish 701 sentences
finish 801 sentences
finish 901 sentences
finish 1001 sentences
finish 1101 sentences
finish 1201 sentences
finish 1301 sentences
finish 1401 sentences
finish 1501 sentences
finish 1601 sentences
finish 1701 sentences
finish 1801 sentences
finish 1901 sentences
finish 2001 sentences
finish 2101 sentences
finish 2201 sentences
finish 2301 sentences
finish 2401 sentences
finish 2501 sentences
finish 2601 sentences
finish 2701 sentences
finish 2801 sentences
finish 2901 sentences
finish 3001 sentences
finish 3101 sentences
finish 3201 sentences
finish 3301 sentences
finish 3401 sentences
finish 3501 sentences
finish 3601 sentences
finish 3701 sentences
finish 3801 sentences
finish 3901 sentences
finish 4001 sentences
finish 4101 sentences
finish 4201 sentences
finish 4301 sentences
finish 4401 sentences
finish 4501 sentences


In [86]:
perfect = []
for j in range(0,len(AiritiRaw)):
    perfect.append([AiritiRaw[j][0],AiritiRaw[j][1],cboson[j],tag_boson[j]])
#wiki_chinese
with open('Airiti.pickle','wb') as f:
    pickle.dump(perfect,f)

In [48]:
#with open('Airiti.pickle','rb') as f:
#    perfect = pickle.load(f)

In [87]:
perfect[11]

['近年來雖然偵測手勢的技術已經被充分的探索',
 ' While high-resolution and miniature gesture sensing technology has been widely explored the interaction space is still limited due to the nature of low resolution human proprioceptive sense',
 '近年來#雖然#偵#測#手勢#的#技術#已經#被#充分#的#探索#',
 'dl#c#v#v#n#ude#n#d#pbei#a#ude#n#']

In [45]:
#import nltk
#import ssl
#try:
#    _create_unverified_https_context = ssl._create_unverified_context
#except AttributeError:
#    pass
#else:
#    ssl._create_default_https_context = _create_unverified_https_context

#nltk.download()

In [88]:
eng = []
for x in range(0,len(perfect)):
    if x%1000==0:
        print(x)
    tmpstr = '<bos>/<sos>'
    words = nltk.word_tokenize(perfect[x][1])
    word_tag = nltk.pos_tag(words)
    for i in range(0,len(word_tag)):
        tmpstr = tmpstr+' '+word_tag[i][0]+'/'+word_tag[i][1]
    tmpstr = tmpstr+' <eos>/<sos>'
    eng.append(tmpstr)

0
1000
2000
3000
4000
5000
6000
7000
8000


In [89]:
chi = []
for x in range(0,len(perfect)):
    a = perfect[x][2].split('#')
    b = perfect[x][3].split('#')
    if(len(a)-len(b))==0:
        tmpstr = '<bos>/<sos>'
        for i in range(0,len(a)-1):
            tmpstr = tmpstr+' '+a[i]+'/'+b[i]
        tmpstr = tmpstr+' <eos>/<sos>'
    else:
        tmpstr = '<bos>/<sos>''<eos>/<sos>'
        print(x,'Wtf')
    chi.append(tmpstr)

In [90]:
with open('0415英文直接對應文本.txt', 'w+') as f:
    for item in eng:
        f.write("%s\n" % item)
        
with open('0415中文直接對應文本.txt', 'w+') as f:
    for item in chi:
        f.write("%s\n" % item)

In [91]:
realc = readfile('0414中文直接對應文本.txt')
reale = readfile('0414英文直接對應文本.txt')

In [92]:
realc[115]

['<bos>/<sos> 任何人/r 都/d 可以/v 透過/v 社群/n 網站/n 傳遞/v 訊息/n 這/r 也/d 造成/v 假/a 新聞/n 的/ude 問題/n 變/v 得/ude 越來越/d 嚴重/ad 想/v 要/v 解決/v 假/a 新聞/n 的/ude 問題/n <eos>/<sos>']

In [81]:
reale[115]

['<bos>/<sos> Recently/RB the/DT fake/JJ news/NN problem/NN becomes/VBZ more/JJR and/CC more/RBR serious/JJ because/IN anyone/NN can/MD tweet/VB or/CC post/VB anything/NN with/IN the/DT click/NN of/IN a/DT mouse/NN <eos>/<sos>']